**Important DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!**

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


## Imports & Setup

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 12:44 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
spark

<pyspark.sql.session.SparkSession at 0x...>

In [6]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'db204905756' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
#         print(b.name)
        paths.append(full_path+b.name)

GCP setup is complete! If you got here without any errors you've earned 10 out of the 35 points of this part.

## Building an inverted index
Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [7]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [8]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called inverted_index_gcp.py, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS):** Use your implementation of word_count, reduce_word_counts, calculate_df, and partition_postings_and_write functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:

- The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
- You need to slightly modify your implementation of partition_postings_and_write because the signature of InvertedIndex.write_a_posting_list has changed and now includes an additional argument called bucket_name for the target bucket. See the module for more details.
- You are not allowed to change any of the code not coming from Colab.

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also"', 'links', 'extrenal',
                 'first', 'see', 'new', 'two', 'list', 'may', 'one', 'district',
                 'including', 'became', 'however', 'com', 'many', 'began',
                 'make', 'made', 'part', 'would', 'people', 'second', 'also',
                 'following', 'history', 'thumb', 'external']

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\'\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def partition_postings_and_write(postings, base_dir):
    bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

    def write_bucket(b_w_pl):
        bucket_id, word_posting_pairs = b_w_pl
        return InvertedIndex.write_a_posting_list((bucket_id, list(word_posting_pairs)), bucket_name)

    posting_locs_list = bucket_rdd.map(write_bucket)

    return posting_locs_list

# הפעלה עבור אינדקס הכותרות (Title)
# וודא ששינית את word_counts לעבוד על doc_title_pairs
_ = partition_postings_and_write(postings, "title").collect()  # ✅ Correct


def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
    return postings.map(lambda token: (token[0], len(token[1])))

def tokenize(text):
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed



def stemmed_word_count(text, id):
    tokens = tokenize(text)
    token_counts = Counter(tokens)
    res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
    return res_list

In [12]:
# time the index creation time
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

In [13]:
# partition posting lists and write out
_ = partition_postings_and_write(postings, "title").collect()

In [14]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):  # Changed from 'title' to 'postings_gcp'
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

## Putting it all together

`gs://{bucket_name}/title_stemmed/{index_src}`

In [15]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/title_stemmed/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 62.6 MiB/ 62.6 MiB]                                                
Operation completed over 1 objects/62.6 MiB.                                     


In [16]:
!gsutil ls -lh $index_dst

 62.61 MiB  2026-01-10T10:27:32Z  gs://db204905756/title_stemmed/index.pkl
TOTAL: 1 objects, 65648392 bytes (62.61 MiB)


In [17]:
title_doc_lengths_dict = {}

def len_tokens(id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (id, len(tokens))

# Apply the function to our RDD
title_len_pairs = doc_title_pairs.map(lambda x: len_tokens(x[1], x[0]))
title_doc_lengths_dict = title_len_pairs.collectAsMap()

# saving the dict as pickle file on the cluster
with open('title_doc_lengths.pickle', 'wb') as f:
    pickle.dump(title_doc_lengths_dict, f)

src = "title_doc_lengths.pickle"
dest = f'gs://{bucket_name}/title_stemmed/{src}'

!gsutil cp $src $dest

Copying file://title_doc_lengths.pickle [Content-Type=application/octet-stream]...
- [1 files][ 42.3 MiB/ 42.3 MiB]                                                
Operation completed over 1 objects/42.3 MiB.                                     


In [18]:
doc_title_pairs.take(20)

[Row(title='Foster Air Force Base', id=4045403),
 Row(title='Torino Palavela', id=4045413),
 Row(title='Mad About the Boy', id=4045419),
 Row(title='Shayne Breuer', id=4045426),
 Row(title='Parantaka I', id=4045432),
 Row(title='Arundel (UK Parliament constituency)', id=4045456),
 Row(title='Andrew Martinez', id=4045466),
 Row(title='Vancouver VooDoo', id=4045471),
 Row(title='Invisible plane', id=4045479),
 Row(title='Shopping channel', id=4045516),
 Row(title='Turgay (river)', id=4045519),
 Row(title='Turgay', id=4045523),
 Row(title='Heinrich Johann Nepomuk von Crantz', id=4045525),
 Row(title='KUT', id=4045532),
 Row(title='Dodge Cove', id=4045544),
 Row(title='Triphone', id=4045546),
 Row(title='Government House (Nova Scotia)', id=4045554),
 Row(title='Bud Abell', id=4045575),
 Row(title='Susette La Flesche', id=4045577),
 Row(title='Pomme de Terre River (Missouri)', id=4045581)]

In [19]:
doc_title_pairs = doc_title_pairs.map(lambda x: (x[1], x[0]))
even_doc_title_pairs = doc_title_pairs.filter(lambda x: x[0] % 2 == 0)
uneven_doc_title_pairs = doc_title_pairs.filter(lambda x: x[0] % 2 != 0)
doc_title_pairs.take(5)

[(4045403, 'Foster Air Force Base'),
 (4045413, 'Torino Palavela'),
 (4045419, 'Mad About the Boy'),
 (4045426, 'Shayne Breuer'),
 (4045432, 'Parantaka I')]

In [20]:
even_id_title = even_doc_title_pairs.collectAsMap()
name = "even_id_title_dict.pkl"
with open(name,'wb') as f:
    pickle.dump(even_id_title, f)
    
bucket = client.bucket(bucket_name)
blob = bucket.blob(f"id_title/{name}")

blob.upload_from_filename(name)

In [21]:
uneven_id_title = uneven_doc_title_pairs.collectAsMap()
name = "uneven_id_title_dict.pkl"
with open(name,'wb') as f:
    pickle.dump(uneven_id_title, f)
    
bucket = client.bucket(bucket_name)
blob = bucket.blob(f"id_title/{name}")

blob.upload_from_filename(name)

In [22]:
even_doc_title_pairs.take(10)

[(4045426, 'Shayne Breuer'),
 (4045432, 'Parantaka I'),
 (4045456, 'Arundel (UK Parliament constituency)'),
 (4045466, 'Andrew Martinez'),
 (4045516, 'Shopping channel'),
 (4045532, 'KUT'),
 (4045544, 'Dodge Cove'),
 (4045546, 'Triphone'),
 (4045554, 'Government House (Nova Scotia)'),
 (4045584, 'Elizabethan Club')]